In [1]:
from sas7bdat import SAS7BDAT
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder
random.seed(1234)
np.random.seed(1234)

In [2]:
with SAS7BDAT('ae.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()
df['AESEV'].value_counts()
df['adverse effect'] = df['AESEV'].apply(lambda x: x.lower()) + ' ' + df['AEDECOD'].apply(lambda x: x.lower())
ae_list = df['adverse effect'].apply(lambda x: x if 'severe'in x else '').value_counts()[1:11].index.tolist()
ae_cols = []
for ae in ae_list:
    ae_name = 'aderse effect: ' + ae
    df[ae_name] = np.zeros(len(df))
    df.loc[df['adverse effect']==ae,ae_name] = 1
    ae_cols.append(ae_name)
df_processed = df[ae_cols+['RUSUBJID']].groupby('RUSUBJID').max().reset_index()
df_processed=df_processed.replace({0:'Yes',1:'No'})
df_processed.shape

(1389, 11)

In [3]:
with SAS7BDAT('cm.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()

In [4]:
drug_list = df['CMDECOD'].value_counts()[:10].index.tolist()
drug_list = [d for d in drug_list if len(d)>0]
drug_name_list = []
drug_list
for drug in drug_list:
    drug_name = 'medication: ' + drug.lower()
    drug_name_list.append(drug_name)
    df[drug_name] = np.zeros(len(df))
    df.loc[df['CMDECOD'] == drug, drug_name] = 1
df_drug = df[drug_name_list+['RUSUBJID']].groupby('RUSUBJID').max().reset_index()
df_drug=df_drug.replace({0:'Yes',1:'No'})
df_drug.shape

(1600, 10)

In [5]:
df_processed = df_drug.merge(df_processed, on='RUSUBJID', how='left')

In [6]:
df_processed.fillna('No', inplace=True)

In [7]:
df_processed.isnull().sum(1).value_counts()

0    1600
dtype: int64

In [8]:
with SAS7BDAT('dm.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()

In [9]:
df_dm = df[['AGEC','SEX','RUSUBJID']]
df_dm = df_dm.rename(columns={'AGEC':'age','SEX':'sex'})
df_processed = df_processed.merge(df_dm, on='RUSUBJID', how='left')

In [10]:
df_processed.isnull().sum(1).value_counts()

0    1600
dtype: int64

In [11]:
with SAS7BDAT('mh.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()
dz_list = df['MHDECOD'].value_counts()[:10].index.tolist()
df['mh_status'] = df['MHOCCUR'].apply(lambda x: 0 if x == 'N' else 1)
dz_name_list = []
for dz in dz_list:
    dz_name = 'historical disease: ' + dz.lower()
    dz_name_list.append(dz_name)
    df[dz_name] = (df['MHDECOD'] == dz) * df['mh_status']
df_dz = df[dz_name_list+['RUSUBJID']].groupby('RUSUBJID').max().reset_index()
df_dz = df_dz.replace({0:'No',1:'Yes'})

In [12]:
df_processed = df_dz.merge(df_processed, on='RUSUBJID', how='left')

In [13]:
df_processed['age'].replace({'>85':'85'},inplace=True)

In [14]:
df_processed['age'].fillna(df_processed['age'].median(),inplace=True)

In [15]:
df_processed['age'] = df_processed['age'].astype(int)

In [16]:
df_processed.fillna('No',inplace=True)
df_processed.isnull().sum(1).value_counts()

0    1604
dtype: int64

In [17]:
with SAS7BDAT('ds.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()

In [18]:
df['DSSCAT'].value_counts()

INFORMED CONSENT                      1604
SCREENING                             1604
ELIGIBILITY FOR RANDOMIZATION         1604
RANDOMIZATION                         1604
END OF TREATMENT                      1604
LAST CONTACT                          1589
SURVIVAL STATUS                       1565
HOSPITALIZATION                       1141
DEATH                                  714
INFORMED CONSENT FOR GENOMIC STUDY     558
CODE BREAKING                            5
Name: DSSCAT, dtype: int64

In [19]:
df['target_label'] = df['DSSCAT'].apply(lambda x:1 if x == 'DEATH' else 0)

In [20]:
df_label = df[['target_label','RUSUBJID']].groupby('RUSUBJID').max().reset_index()

In [21]:
df_processed = df_processed.merge(df_label, on='RUSUBJID')

In [22]:
df_processed.head()

,RUSUBJID,historical disease: deep vein thrombosis,historical disease: pulmonary embolism,historical disease: antiandrogen therapy,historical disease: cardiac failure chronic,historical disease: chronic respiratory failure,historical disease: venous insufficiency,historical disease: coronary artery disease,historical disease: myocardial infarction,historical disease: hypertension,...,aderse effect: severe anaemia,aderse effect: severe vomiting,aderse effect: severe pneumonia,aderse effect: severe diarrhoea,aderse effect: severe abdominal pain,aderse effect: severe sepsis,aderse effect: severe leukopenia,age,sex,target_label
0,006521-000-901-000,No,No,No,No,No,No,No,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,67,M,0
1,006521-000-901-001,No,No,No,No,No,No,No,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,67,M,1
2,006521-000-901-002,No,No,No,No,No,No,No,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,51,M,0
3,006521-000-901-003,No,No,No,No,No,No,No,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,49,M,1
4,006521-000-901-004,No,No,No,No,No,No,No,No,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,64,F,1


In [23]:
with SAS7BDAT('lb.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()

In [24]:
df = df[df['LBBLFL'] == 'Y']

In [25]:
lb_list = df['LBTEST'].value_counts()[:10].index.tolist()
lb_name_list = []

In [26]:
for lb in lb_list:
    lb_name = 'lab test: ' + lb.lower()
    lb_name_list.append(lb_name)
    df_lb = df[df['LBTEST'] == lb][['LBSTRESN','RUSUBJID']].groupby('RUSUBJID').mean().reset_index()
    df_lb = df_lb.rename(columns = {'LBSTRESN':lb_name})
    df_processed = df_processed.merge(df_lb, on='RUSUBJID', how='left')

In [27]:
df_lb = df[df['LBTEST'] == lb][['LBSTRESN','RUSUBJID']].groupby('RUSUBJID').mean().reset_index()

In [28]:
df_processed = df_processed.fillna(df_processed.median())

C:\Users\ZF\AppData\Local\Temp/ipykernel_17868/822859786.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_processed = df_processed.fillna(df_processed.median())


In [29]:
numerical_features = ['age'] + lb_name_list
f = open('NCT00694382_numerical_feature.txt','w')
for x in numerical_features: f.write(x+'\n')
f.close()

In [30]:
binary_features = dz_name_list + ae_cols + drug_name_list
f = open('NCT00694382_binary_feature.txt','w')
for x in binary_features: f.write(x+'\n')
f.close()

In [31]:
df_processed = df_processed.rename(columns={'RUSUBJID':'patient_id'})
df_processed = df_processed.drop('patient_id', axis=1)

In [32]:
df_processed.to_csv('NCT00694382.csv')

# Process inc/exc criteria

In [33]:
with SAS7BDAT('ie.sas7bdat', skip_header=False) as reader:
    df_ie = reader.to_data_frame()

In [34]:
with SAS7BDAT('ti.sas7bdat', skip_header=False) as reader:
    df_ti = reader.to_data_frame()

In [35]:
with SAS7BDAT('dm.sas7bdat', skip_header=False) as reader:
    df_dm = reader.to_data_frame()

In [36]:
inc_exc_columns = []
for c in df_ti['IETESTCD'].values:
    if 'E' in c: df_dm[c] = np.zeros(len(df_dm))
    elif 'I' in c: df_dm[c] = np.ones(len(df_dm))
    inc_exc_columns.append(c)
for row in df_ie.index:
    subjid = df_ie.loc[row,'RUSUBJID']
    ietestcd = df_ie.loc[row, 'IETESTCD']
    if 'E' in ietestcd: df_dm.loc[df_dm['RUSUBJID'] == subjid, ietestcd] = 1
    elif 'I' in ietestcd: df_dm.loc[df_dm['RUSUBJID'] == subjid, ietestcd] = 0
df_inc_exc = df_dm[inc_exc_columns+['RUSUBJID']].rename(columns={'RUSUBJID':'patient_id'})
df_inc_exc.to_csv('NCT00694382_inc_exc.csv')

In [37]:
# Process for TransTab
df_processed

,historical disease: deep vein thrombosis,historical disease: pulmonary embolism,historical disease: antiandrogen therapy,historical disease: cardiac failure chronic,historical disease: chronic respiratory failure,historical disease: venous insufficiency,historical disease: coronary artery disease,historical disease: myocardial infarction,historical disease: hypertension,historical disease: peripheral arterial occlusive disease,...,lab test: hemoglobin,lab test: leukocytes,lab test: creatinine clearance,lab test: creatinine,lab test: platelet,lab test: bilirubin,lab test: alanine aminotransferase,lab test: aspartate aminotransferase,lab test: neutrophils,lab test: alkaline phosphatase
0,No,No,No,No,No,No,No,No,No,No,...,138.000,7.67,65.865411,90.000,341.0,8.100,8.00000,11.00000,6.78,102.0000
1,No,No,No,No,No,No,No,No,No,No,...,132.000,4.90,114.308316,62.764,244.0,17.300,107.97840,86.38272,3.47,102.0000
2,No,No,No,No,No,No,No,No,No,No,...,122.436,5.50,125.144312,83.000,197.0,5.300,13.19736,20.99580,4.10,107.0000
3,No,No,No,No,No,No,No,No,No,No,...,139.000,9.80,113.245477,77.000,343.0,5.700,16.00000,17.00000,8.20,54.0000
4,No,No,No,No,No,No,No,No,Yes,No,...,108.000,8.60,66.464988,80.000,382.0,6.800,20.39592,35.99280,6.36,212.9574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,No,No,No,No,No,No,No,No,No,No,...,171.000,12.50,84.727314,82.212,245.0,8.892,10.00000,25.00000,11.30,66.0000
1600,No,No,No,No,No,No,No,No,No,Yes,...,117.000,9.60,46.461916,100.000,387.0,5.000,16.00000,24.00000,6.40,77.0000
1601,No,No,No,No,No,No,No,No,Yes,No,...,110.000,3.08,65.316242,93.704,283.0,7.182,22.00000,32.00000,1.61,209.0000
1602,No,No,No,No,No,Yes,Yes,No,Yes,No,...,123.000,3.00,51.441872,95.000,255.0,16.600,13.90000,18.20000,2.19,82.5000


In [38]:
df_processed[drug_name_list]

,medication: dexamethasone,medication: ondansetron,medication: heparin,medication: fluorouracil,medication: ranitidine,medication: cisplatin,medication: metoclopramide,medication: carboplatin,medication: furosemide
0,No,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes
1,No,Yes,Yes,Yes,Yes,No,Yes,Yes,No
2,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes
3,Yes,Yes,Yes,Yes,Yes,No,Yes,No,Yes
4,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes
...,...,...,...,...,...,...,...,...,...
1599,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes
1600,No,No,Yes,Yes,Yes,Yes,No,No,Yes
1601,No,No,Yes,No,No,No,Yes,Yes,Yes
1602,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes
